In [83]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd

In [87]:
class FullyConnectedNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FullyConnectedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.Tanh()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        return out

# Подготовка данных для обучения
input_size = 64 * 64  # Размер изображения
hidden_size = 512  # Количество нейронов в скрытом слое
output_size = 1  # Размер выходного слоя

In [93]:
# Подготовка данных (пример)
X = pd.read_csv(r'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\X_train_2.csv')
y = pd.read_csv(r'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\y_train_2.csv')
# Предполагается, что данные содержат признаки в столбцах и целевую переменную в последнем столбце

# Разделение данных на признаки и целевую переменную
X = X.iloc[:, :-1].values
y = y.iloc[:, -1].values  # Если нужно преобразовать вектор в столбец

# Преобразование в тензоры и перемещение на GPU
X_tensor = torch.tensor(X, dtype=torch.float32).cuda()
y_tensor = torch.tensor(y, dtype=torch.float32).cuda()

# Определение модели и перемещение на GPU
model = FullyConnectedNN(input_size, hidden_size, output_size).cuda()

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()  # Функция потерь для регрессии
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Оптимизатор Adam с learning rate 0.001

# Обучение модели
num_epochs = 2
for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(len(X_tensor)):
        inputs = X_tensor[i]
        labels = y_tensor[i]
        
        # Обнуление градиентов параметров
        optimizer.zero_grad()
        
        # Прямой проход
        outputs = model(inputs)
        
        # Рассчет функции потерь
        loss = criterion(outputs, labels)
        
        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(X_tensor)}")

print("Обучение завершено")

C:\PycharmProjects\ETS_Autopilot\venv_3_11\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/2], Loss: 0.0033439287721135002
Epoch [2/2], Loss: 0.0028652651049211595
Обучение завершено


In [95]:
model.forward(X_tensor[230])

tensor([0.0151], device='cuda:0', grad_fn=<TanhBackward0>)

In [ ]:
model.forward(X_tensor[620])

In [96]:
torch.save(model.state_dict(), 'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\weight_1.pth')

In [ ]:
import vgamepad as vg
import time

# Инициализация виртуального геймпада
gamepad = vg.VX360Gamepad()

# Функция для установки значения оси "руля"
def set_wheel_axis(value):
    gamepad.axis(vg.XUSB_AXIS.RX, value)
    gamepad.update()

set_wheel_axis(50)


gamepad.stop()